In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import random
import scipy.optimize

1. Create a Data class to load and manipulate the data.
2. Create functions to do a data analysis, calculate similarity using Jaccard similarity.
3. Make a recommendation for a selected item.

In [2]:
class Data:
    def __init__(self):
        '''init Data class'''
        self.data = None
                
    def load_data(self, filename, format='txt'):
        '''loads data from excel, csv, tsv, or txt file'''
        if format == 'excel':
            self.data = pd.read_excel(filename)
        elif format == 'csv':
            self.data = pd.read_csv(filename)
        elif format == 'tsv':
            self.data = pd.read_csv(filename, sep='\t')
        elif format == 'txt':
            self.data = pd.read_table(filename)
        else:
            raise ValueError('Invalid file format.  Please specify "excel", "csv", "tsv", or "txt".')
          
    def present_data_head(self, number_rows=2):
        '''present information from the data'''
        display(self.data.head(number_rows))
        self.data.info()
    
    def drop_small_order(self, order_col = 'order_number', min_order_size = 2):
        '''delete small orders that have less number of items or smaller than a threshold'''
        self.data = self.data[self.data.groupby('order_number').order_number.transform(len) >= min_order_size]
        
    def drop_columns(self,columns = []):
        '''drop column from self.data'''
        self.data.drop(columns, axis = 1, inplace = True)

In [3]:
data = Data()
data.load_data('C:/Users/Son/Google Drive/DSDJ/DSDJ/Projects/E-Commerce/data/All Transations - 2 Weeks.txt', format='txt')
print("\n Orgiginal data")
data.present_data_head(number_rows=10)
# remove the orders that have less than 10 items
data.drop_small_order(order_col = 'order_number', min_order_size = 10)
print("\n Data after eleminating small orders \n")
data.present_data_head(number_rows=5)



 Orgiginal data


,order_number,l1,l2,l3,sku,brand
0,168266,Power Tools,Power Saws and Accessories,Reciprocating Saw Blades,265105,2768
1,123986,Safety,Spill Control Supplies,Temporary Leak Repair,215839,586
2,158978,Hardware,Door Hardware,Thresholds,284756,1793
3,449035,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,12579,1231
4,781232,Motors,General Purpose AC Motors,General Purpose AC Motors,194681,2603
5,116599,Pneumatics,Pneumatic Tube Fittings,Pneumatic Push to Connect Tube Fittings,167757,3889
6,701116,Motors,General Purpose AC Motors,General Purpose AC Motors,310296,1068
7,555497,Motors,Motor Supplies,Capacitors,306732,1068
8,282317,Safety,Footwear and Footwear Accessories,Insoles,148549,2696
9,644437,Hand Tools,Sockets and Bits,Crowfoot Socket Wrenches,283869,3356


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2107537 entries, 0 to 2107536
Data columns (total 6 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   order_number  int64 
 1   l1            object
 2   l2            object
 3   l3            object
 4   sku           int64 
 5   brand         int64 
dtypes: int64(3), object(3)
memory usage: 96.5+ MB

 Data after eleminating small orders 



,order_number,l1,l2,l3,sku,brand
22,173984,Cleaning,Paper Products and Dispensers,"Paper Towels, Rolls",46841,1726
37,649979,Power Tools,Drilling Accessories,Step Drill Bits,299689,2163
41,565917,"Paint, Equipment and Supplies",Spray Paints and Primers,Spray Paints,310773,3612
58,482732,Hand Tools,Marking Tools,Paint Markers,38789,4477
64,173984,Cleaning,Floor Care,Floor Finishes and Sealers,264751,3712


<class 'pandas.core.frame.DataFrame'>
Int64Index: 551452 entries, 22 to 2107536
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   order_number  551452 non-null  int64 
 1   l1            551452 non-null  object
 2   l2            551452 non-null  object
 3   l3            551452 non-null  object
 4   sku           551452 non-null  int64 
 5   brand         551452 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 29.5+ MB


In [4]:
# convert data frame into a dict and present the first one
dataset = data.data.to_dict('r')
dataset[0]

C:\Users\Son\anaconda3\lib\site-packages\pandas\core\frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


{'order_number': 173984,
 'l1': 'Cleaning',
 'l2': 'Paper Products and Dispensers',
 'l3': 'Paper Towels, Rolls',
 'sku': 46841,
 'brand': 1726}

In [5]:
# Build 02 set:
# usersPerItem: which users purchased item
# itemsPerUser: which items is purchased by each user
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)

In [6]:
# Build dictionary of itemNames
itemNames = {}
itemBrands = {}

In [7]:
# go through the dataset
# extract order number and item ID
# add user (order number) to corresponding users
# add item (item ID) to corresponding items
for d in dataset:
    user, item = d['order_number'], d['sku']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    itemNames[item] = d['l3']
    itemBrands[item] = d['brand']
#print("usersPerItem: \n", usersPerItem[1])    
#print("itemsPerUser: \n", itemsPerUser[1])    

In [8]:
# create a Jaccard similarity function
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    else:
        return numer / denom

In [9]:
# create a function that display items' names and their brand IDs
def Display_itemName_brandID(rec):
    dis = []
    for y in rec:
        dis.append((itemNames[y[1]], itemBrands[y[1]]))
    return dis

1. Find a set of users who purchased i
2. Iterate over all users who purchased i
3. Build a candidate set from all items those users comsumed
4. For items in this set, compute their similarity with i (and store it)
5. Sort all the items by (Jaccard) similarity
6. Return the most similar

In [10]:
def mostsimilarFast(i):
    similarities = []
    users = usersPerItem[i]
    candidateItems = set()
    for u in users:
        candidateItems = candidateItems.union(itemsPerUser[u])
    for i2 in candidateItems:
        if i2 ==i: continue
        sim = Jaccard(users,usersPerItem[i2])
        similarities.append((sim,i2))
    similarities.sort(reverse=True)
    return similarities[:5]

Select one item and recommend top 5 most similar items


In [11]:
#select 01 data point, 
dataset[4]
print("Display 01 data point:")
display(dataset[4])

#take item ID
query = dataset[4]['sku']
# print item name
print(" Item name: \n",itemNames[query], itemBrands[query])

mostsimilarFast(query)
print("\n Item IDs and their associated Jaccard similarity values:")
display(mostsimilarFast(query))

print("Items that are recommended:")
#[itemNames[y[1]] for y in mostsimilarFast(query)]
Display_itemName_brandID(mostsimilarFast(query))

Display 01 data point:


{'order_number': 173984,
 'l1': 'Cleaning',
 'l2': 'Floor Care',
 'l3': 'Floor Finishes and Sealers',
 'sku': 264751,
 'brand': 3712}

 Item name: 
 Floor Finishes and Sealers 3712

 Item IDs and their associated Jaccard similarity values:


[(0.5, 37462),
 (0.3333333333333333, 243083),
 (0.3333333333333333, 51094),
 (0.16666666666666666, 165562),
 (0.125, 189128)]

Items that are recommended:


[('Hand Sanitizer, Lotion, and Soap Dispensers', 1767),
 ('Toilet Paper Dispensers', 1726),
 ('Dust Mops', 4355),
 ('Surface and Air Deodorants', 3008),
 ('Toilet Bowl Cleaners', 4815)]

1. Select randomly three items
2. Display item IDs and item names
3. For each item, two most similarity items are recommended
4. Display all recommended item IDs, their Jaccard similarity values
5. Display recommended item names

In [12]:
# select randomly three items
three=data.data['sku'].sample(n=3,replace=True)
three.reset_index(drop=True, inplace=True)
print("Item IDs:")
display(three)

# display item names
print("Item names and brand IDs:")
for i in three.index:
    print(itemNames[three[i]], itemBrands[three[i]])

# for each item, call funtion mostsimilarFast
# top two similar items are recorded
groups = []
for i in three.index:
    groups.extend(mostsimilarFast(three[i])[:2])
#groups.sort(reverse=True)

print("\nRecommended item IDs and their Jaccard similarity values:")
display(groups[:6])
print("Recommended item names and their brand IDs:")
#[itemNames[y[1]] for y in groups[:6]]  
Display_itemName_brandID(groups[:6])

Item IDs:


0    279312
1     79915
2     14439
Name: sku, dtype: int64

Item names and brand IDs:
Fork Terminals 9
Flat Washers 1793
Cleaning Rags 4355

Recommended item IDs and their Jaccard similarity values:


[(0.16666666666666666, 294961),
 (0.16666666666666666, 288303),
 (0.15555555555555556, 83171),
 (0.12359550561797752, 79912),
 (0.08695652173913043, 268111),
 (0.08, 51123)]

Recommended item names and their brand IDs:


[('Ring Terminals', 3271),
 ('Thumb Screws', 1793),
 ('Lock Washers', 1793),
 ('Flat Washers', 1793),
 ('Trash Bags', 4355),
 ('Dust Mop Kits', 4355)]